In [1]:
!pip install pytorch_lightning #torch==1.3.1
!pip install transformers
!pip3 -q install emoji


In [2]:
%matplotlib inline

import json
import logging
from pathlib import Path
import random
import tarfile
import tempfile
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import pandas_path  # Path style access for pandas
from tqdm import tqdm

import torch                    
import torchvision
import fasttext
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [3]:
import ast
import pandas as pd

def load_data_lists(path):
    data_points_lists = []
    with open(path, encoding='utf-8') as f:
        lines = f.readlines()

        for line in lines:
            try:
                data_points_lists.append(ast.literal_eval(line))
            except:
                # Ignore lines with errors
                pass

    print('Found {} lines in "{}".'.format(len(lines), path))
    print('Successfully loaded {} data points from "{}".'.format(len(data_points_lists), path))
    
    return data_points_lists

COLUMN_NAMES = ['ID', 'Text', 'Sarcastic']

def construct_df(data_points_lists, column_names=COLUMN_NAMES):
    df = pd.DataFrame(data_points_lists, columns=column_names)
    df['ID'] = pd.to_numeric(df['ID'])
    df['Sarcastic'] = df['Sarcastic'].astype('bool')
    
    return df

In [4]:
df_train = construct_df(load_data_lists('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/train.txt'))

print(df_train.shape)
print(df_train.info())

display(df_train.head())

Found 29040 lines in "/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/train.txt".
Successfully loaded 29040 data points from "/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/train.txt".
(29040, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29040 entries, 0 to 29039
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         29040 non-null  int64 
 1   Text       29040 non-null  object
 2   Sarcastic  29040 non-null  bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 482.2+ KB
None


,ID,Text,Sarcastic
0,910308516510011393,most # funny quotes : 21 snarky and # funny qu...,True
1,725333760762363905,spurs # creativethinking ! <url>,True
2,840006160660983809,<user> thanks for showing up for our appointme...,True
3,854334602516733952,only a hardcore fan of sir jonny sins will get...,True
4,908913372199915520,haha . # lol,True


In [5]:
df_val = construct_df(load_data_lists('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/valid.txt'), COLUMN_NAMES + ['Sarcastic_human'])
print(df_val.shape)
print(df_val.info())

display(df_val.head())

Found 2410 lines in "/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/valid.txt".
Successfully loaded 2410 data points from "/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/valid.txt".
(2410, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               2410 non-null   int64 
 1   Text             2410 non-null   object
 2   Sarcastic        2410 non-null   bool  
 3   Sarcastic_human  2410 non-null   int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 59.0+ KB
None


,ID,Text,Sarcastic,Sarcastic_human
0,915657464401580032,whew ... that extra <num> miles today to the g...,True,1
1,854678856724340736,""" oh , good . now no one will know we 're here...",True,1
2,904892917277274112,how much of it you think is true ? has this be...,True,1
3,855466461296504832,<user> finally found proof that the earth is f...,True,1
4,927373534652805120,many ways to overcome tension & fear but nothi...,True,1


In [6]:
df_test = construct_df(load_data_lists('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/test.txt'), COLUMN_NAMES + ['Sarcastic_human'])
print(df_test.shape)
print(df_test.info())

display(df_test.head())

Found 2409 lines in "/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/test.txt".
Successfully loaded 2409 data points from "/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/text_data/test.txt".
(2409, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409 entries, 0 to 2408
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               2409 non-null   int64 
 1   Text             2409 non-null   object
 2   Sarcastic        2409 non-null   bool  
 3   Sarcastic_human  2409 non-null   int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 58.9+ KB
None


,ID,Text,Sarcastic,Sarcastic_human
0,862902619928506372,i am guessing # netflix no longer lets you gra...,True,1
1,892551658487631873,it 's the insensitive strikeouts at suntrust p...,True,1
2,853143461360480256,"following the path of the river calder , so .....",True,1
3,918423568823840768,# westernsahara # authority has no lessons 2ge...,True,1
4,731617467718610944,hey <user> great sale !,True,1


In [7]:
# Create subdirectories
import os
import shutil

train_dir = '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/'
val_dir = '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val/'
test_dir = '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/test/'

os.mkdir(train_dir)
os.mkdir(val_dir)
os.mkdir(test_dir)

In [8]:
missing_ids = []

for id in df_train['ID']:
    img_path = '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/{}.jpg'.format(id)
    #img_path = 'dataset_image/{}.jpg'.format(id)
    if os.path.isfile(img_path):
        #shutil.move(img_path, '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/{}.jpg'.format(id))

      shutil.move(img_path, '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/{}.jpg'.format(id))
    else:
        missing_ids.append(id)

In [9]:
len(missing_ids)


9224

In [10]:
df_train = df_train[~df_train['ID'].isin(missing_ids)]
df_train.shape

(19816, 3)

In [11]:
for id in df_val['ID']:
    img_path = '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/{}.jpg'.format(id)
    if os.path.isfile(img_path):
      shutil.move(img_path, '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val/{}.jpg'.format(id))

In [12]:
for id in df_test['ID']:
    img_path = '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/{}.jpg'.format(id)
    if os.path.isfile(img_path):
      shutil.move(img_path, '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/test/{}.jpg'.format(id))

In [13]:
os.mkdir('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/sarc/')
os.mkdir('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/non-sarc/')
os.mkdir('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val/sarc/')
os.mkdir('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val/non-sarc/')
os.mkdir('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/test/sarc/')
os.mkdir('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/test/non-sarc/')

In [14]:
class_foldername = {
    True: 'sarc',
    False: 'non-sarc'
}

In [15]:
# Move images to subfolders
for id,label in zip(df_train['ID'], df_train['Sarcastic']):
    shutil.move('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/{}.jpg'.format(id), '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train/{}/{}.jpg'.format(class_foldername[label], id))

In [16]:
# Move images to subfolders
for id,label in zip(df_val['ID'], df_val['Sarcastic']):
    shutil.move('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val/{}.jpg'.format(id), '/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val/{}/{}.jpg'.format(class_foldername[label], id))


In [17]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    datasets.ImageFolder('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/train', data_transforms['train']),
    'validation': 
    datasets.ImageFolder('/Users/albert/Documents/CSE8813_DLT/Sarcasm-Detection-hierarchical_fusion_model/image_data/val', data_transforms['validation'])
}

image_dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True),
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=64,
                                shuffle=False)
}

In [18]:
# dataloader for txt data
from transformers import AutoModel, AutoTokenizer

from sklearn.utils import shuffle
from transformers import (get_linear_schedule_with_warmup,AdamW,AutoModel, AutoTokenizer,
                            AutoModelForSequenceClassification)
from torch.utils.data import (TensorDataset,DataLoader,
                             RandomSampler, SequentialSampler, Dataset)


train_df = shuffle(df_train, random_state=42)
valid_df = shuffle(df_val, random_state=42)
test_df = shuffle(df_test, random_state=42)


In [19]:
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

text_model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/bertweet-base",
        num_labels = 2,  
        output_attentions = False, 
        output_hidden_states = True,
    )

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weig

In [20]:
def bert_encode(df, tokenizer):
    input_ids = []
    attention_masks = []
    # print(df)
    for sent in df[['Text']].values:
        sent = sent.item()
        encoded_dict = tokenizer.encode_plus(
                            sent,                      
                            add_special_tokens = True, 
                            max_length = 128,           
                            pad_to_max_length = True,
                            truncation = True,
                            return_attention_mask = True,   
                            return_tensors = 'pt',    
                    )
           
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    inputs = {
    'input_word_ids': input_ids,
    'input_mask': attention_masks}

    return inputs

In [21]:
from torch.utils.data import (TensorDataset,DataLoader,
                             RandomSampler, SequentialSampler, Dataset)

def prepare_dataloaders(train_df,test_df,batch_size=64):
    # Load the AutoTokenizer with a normalization mode if the input Tweet is raw
    
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
    
    tweet_train = bert_encode(train_df, tokenizer)
    tweet_train_labels = train_df['Sarcastic'].astype(int)
    
    tweet_test = bert_encode(test_df, tokenizer)

    input_ids, attention_masks = tweet_train.values()
    labels = torch.tensor(tweet_train_labels.values)
    train_dataset = TensorDataset(input_ids, attention_masks, labels)

    
    input_ids, attention_masks = tweet_test.values()
    test_dataset = TensorDataset(input_ids, attention_masks)

    
    train_dataloader = DataLoader(
                train_dataset,
                sampler = RandomSampler(train_dataset), 
                batch_size = batch_size 
            )


    test_dataloader = DataLoader(
                test_dataset, 
                sampler = SequentialSampler(test_dataset), 
                batch_size = batch_size
            )
    return train_dataloader, test_dataloader

In [22]:
text_train_dataloader,text_test_dataloader = prepare_dataloaders(train_df, test_df)
_,text_val_dataloader = prepare_dataloaders(train_df, valid_df)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/Users/albert/opt/anaconda3/envs/DLT_project2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [24]:
class CNN_LSTM(nn.Module):
  def __init__(self, vocab_size, weights_matrix, n_hidden, n_layers, n_out):
    super(CNN_LSTM, self).__init__()

    # LSTM for the text overview
    self.vocab_size, self.n_hidden, self.n_out, self.n_layers = vocab_size, n_hidden, n_out, n_layers
    num_embeddings, embedding_dim = weights_matrix.shape[0], weights_matrix.shape[1]
    self.emb = nn.Embedding(num_embeddings, embedding_dim)
    self.emb.weight.data.copy_(torch.from_numpy(weights_matrix))
    self.emb.weight.requires_grad = True
    self.lstm = nn.LSTM(embedding_dim, self.n_hidden, self.n_layers, dropout=0.2, batch_first=True)
    self.dropout = nn.Dropout(0.1)
    self.lstm_fc = nn.Linear(self.n_hidden, 128)
    # self.sigmoid = nn.Sigmoid()

    # CNN for the posters
    self.conv1 = nn.Conv2d(3, 32, 3)
    self.max_pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(32, 64, 3)
    self.max_pool2 = nn.MaxPool2d(2)
    self.conv3 = nn.Conv2d(64, 128, 3)
    self.max_pool3 = nn.MaxPool2d(2)
    self.conv4 = nn.Conv2d(128, 128, 3)
    self.max_pool4 = nn.MaxPool2d(2)
    self.cnn_dropout = nn.Dropout(0.1)
    self.cnn_fc = nn.Linear(5*2*128, 512)
    # Concat layer for the combined feature space
    self.combined_fc1 = nn.Linear(640, 256)
    self.combined_fc2 = nn.Linear(256, 128)
    self.output_fc = nn.Linear(128, n_out)


  def forward(self, lstm_inp, cnn_inp):
    batch_size = lstm_inp.size(0)
    hidden = self.init_hidden(batch_size)
    lstm_inp = lstm_inp.long()
    embeds = self.emb(lstm_inp)
    lstm_out, hidden = self.lstm(embeds, hidden)
    lstm_out = self.dropout(lstm_out[:, -1])
    lstm_out = F.relu(self.lstm_fc(lstm_out))

    x = F.relu(self.conv1(cnn_inp))
    x = self.max_pool1(x)
    x = F.relu(self.conv2(x))
    x = self.max_pool2(x)
    x = F.relu(self.conv3(x))
    x = self.max_pool3(x)
    x = F.relu(self.conv4(x))
    x = self.max_pool4(x)
    x = x.view(-1, 5*2*128)
    x = self.cnn_dropout(x)
    cnn_out = F.relu(self.cnn_fc(x))
    combined_inp = torch.cat((cnn_out, lstm_out), 1)
    x_comb = F.relu(self.combined_fc1(combined_inp))
    x_comb = F.relu(self.combined_fc2(x_comb))
    out = torch.sigmoid(self.output_fc(x_comb))

    return out

  def init_hidden(self, batch_size):
    weight = next(self.parameters()).data
    hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device))
    return hidden

In [25]:
#epoch = 20
#model.train()
#for i in range(epochs):
#  total_acc_train = 0
#  total_loss_train = 0
    
#  for lstm, cnn in zip(text_train_loader, img_train_loader):
#    lstm_inp, lstm_labels = lstm
#    cnn_inp, cnn_labels = cnn
#    lstm_inp, lstm_labels = lstm_inp.to(device), lstm_labels.to(device)
#    cnn_inp, cnn_labels = cnn_inp.to(device), cnn_labels.to(device)
#    model.zero_grad()
#    output = model(lstm_inp, cnn_inp)
#    loss = criterion(output.squeeze(), lstm_labels.float())
#    loss.backward()
#    nn.utils.clip_grad_norm_(model.parameters(), clip)
#    optimizer.step()
    
#    with torch.no_grad():
#      acc = torch.abs(output.squeeze() - lstm_labels.float()).view(-1)
#      acc = (1. - acc.sum() / acc.size()[0])
#      total_acc_train += acc
#      total_loss_train += loss.item()
  
#  train_acc = total_acc_train/len(text_train_loader)
#  train_loss = total_loss_train/len(text_train_loader)
#  model.eval()
#  total_acc_val = 0
#  total_loss_val = 0
#  with torch.no_grad():
#    for lstm, cnn in zip(text_val_loader, img_val_loader):
#      lstm_inp, lstm_labels = lstm
#      cnn_inp, cnn_labels = cnn
#      lstm_inp, lstm_labels = lstm_inp.to(device), lstm_labels.to(device)
#      cnn_inp, cnn_labels = cnn_inp.to(device), cnn_labels.to(device)
#      model.zero_grad()
#      output = model(lstm_inp, cnn_inp)
#      val_loss = criterion(output.squeeze(), lstm_labels.float())
#      acc = torch.abs(output.squeeze() - lstm_labels.float()).view(-1)
#      acc = (1. - acc.sum() / acc.size()[0])
#      total_acc_val += acc
#      total_loss_val += val_loss.item()
#    print("Saving model...") 
#    torch.save(model.state_dict(), dropout.pt')

#  val_acc = total_acc_val/len(text_val_loader)
#  val_loss = total_loss_val/len(text_val_loader)
#  print(f'Epoch {i+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
#  model.train()

In [26]:
def prepare_model(model_class="vinai/bertweet-base",num_classes=2,model_to_load=None,total_steps=-1):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_class,
        num_labels = num_classes,  
        output_attentions = False, 
        output_hidden_states = True,
    )

    optimizer = AdamW(model.parameters(),
                    lr = 5e-5,
                    eps = 1e-8
                    )
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, 
                                                num_training_steps = total_steps)

    if model_to_load is not None:
        try:
            model.roberta.load_state_dict(torch.load(model_to_load))
            print("LOADED MODEL")
        except:
            pass
    return model, optimizer, scheduler

In [27]:
class MultiModel(nn.Module):
    def __init__(self):
        super(MultiModel, self).__init__()
        #Resnet50 for image
        self.text_model, _, _ = prepare_model("vinai/bertweet-base" ,num_classes=2, model_to_load=None, total_steps = total_steps)
        self.linear_1 = nn.Linear(768,300)
        #self.text_model = AutoModelForSequenceClassification.from_pretrained(
        #model_class = "vinai/bertweet-base",
        #num_labels = 2,  
        #output_attentions = False, 
        #output_hidden_states = False,)
        
        self.image_model = torchvision.models.resnet50(
            pretrained=True
        )
        #resnet_output_feature_dim = 300
        #bert_output_feature_dim = 300
        #fusion_input_feature = 300+300 = 600
        self.image_model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 300))
        
        
        # Combined
        
        #fusion_input_feature = 300+300 = 600
        #fusion_output_size = 128
        self.combined_fc1 = nn.Linear(600, 256)
        self.combined_fc2 = nn.Linear(256, 128)
        
        self.output_fc = nn.Linear(128, 2)
       
        #self.text_train_dataset = image_dataloaders["train"]
        #self.text_dev_dataset = image_dataloaders["validation"]
        #self.image_train_dataset = text_train_dataloader
        #self.image_dev_dataset = text_val_dataloader

    
    def forward(self, text,text_mask,image):
        text = text
        bert_out = self.text_model(text)
        last_layer = bert_out
        #print(len(bert_out))
        
        last_layer = bert_out.hidden_states[-1][:, 0, :]
        print(last_layer.shape)
        image = image
        resnet_out = self.image_model(image)
        print("resnet output")
        print(resnet_out.shape)
        last_layer= self.linear_1(last_layer)
        combined_emb = torch.cat((resnet_out, last_layer), 1)
        print("combined_emb")
        print(combined_emb.shape)
        x_comb = F.relu(self.combined_fc1(combined_emb))
        x_comb = F.relu(self.combined_fc2(x_comb))
        out = torch.sigmoid(self.output_fc(x_comb))
        #logits = self.output_fc(x_comb)
        #pred = torch.nn.functional.softmax(logits)
        return out


In [28]:
#count = 0
#for batch, [image,label] in zip(text_train_dataloader,image_dataloaders["train"]):
#    count+=1
 #   print(batch[0].shape)
  #  print(batch[1].shape)
   # print(batch[2])
    #print(image.shape)
    #print(label.shape)
#    if count==5:
#        break

In [29]:
def train_model(model, image_data_loaders,text_train_data_loader,text_val_data_loader, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

        
            for batch,[image_inputs,image_labels] in zip(text_train_data_loader,image_data_loaders[phase]):
                text_inputs = batch[0]
                text_mask = batch[1]
                text_labels = batch[2]
                text_inputs = text_inputs.to(device)
                text_labels = text_labels.to(device)
                image_inputs = image_inputs.to(device)
                image_labels = image_labels.to(device)
                outputs = model(text_inputs,text_mask,image_inputs)
                
                loss = criterion(outputs, image_labels)
                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [30]:
epochs = 10
total_steps = len(train_df) * epochs
model = MultiModel().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(),
                    lr = 5e-5,
                    eps = 1e-8
                    )

#loss_func = nn.MSELoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

In [ ]:
model_trained = train_model(model,image_dataloaders,text_train_dataloader,text_val_dataloader, criterion, optimizer, num_epochs=10)


Epoch 1/10
----------
torch.Size([64, 768])
resnet output
torch.Size([64, 300])
combined_emb
torch.Size([64, 600])


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
torch.save(model, '/content/drive/MyDrive/Sarcasm-Detection-hierarchical_fusion_model/multimodal_10epoch.pt')
